In [1]:
import tensorflow as tf
tf.set_random_seed(7)
print ("TensorFlow version: " + tf.__version__)
import keras

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.5.0


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [2]:
from mlxtend.data import loadlocal_mnist
from sklearn.model_selection import train_test_split
import numpy as np

files = {
    "X": "data/train-images-idx3-ubyte",
    "y": "data/train-labels-idx1-ubyte",
    "X_test": "data/t10k-images-idx3-ubyte",
    "y_test": "data/t10k-labels-idx1-ubyte"
}

# Code to load the binary files.
# https://github.com/rasbt/mlxtend/blob/master/mlxtend/data/local_mnist.py

X, y = loadlocal_mnist(images_path = files['X'], labels_path = files['y'])
X_test, y_test = loadlocal_mnist(images_path = files['X_test'], labels_path = files['y_test'])

### Reshape the data

In [3]:
X = X.reshape(X.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(45000, 28, 28, 1)
(15000, 28, 28, 1)
(10000, 28, 28, 1)


### One-hot Encoding

In [4]:
import numpy as np

def one_hot_encode(data):
    encoded_data = np.zeros((data.shape[0], 10), dtype=np.float32)
    for index, digit in enumerate(data):
        encoded_data[(index, digit)] = 1
    return encoded_data

    
encoded_y_train = one_hot_encode(y_train)
encoded_y_valid = one_hot_encode(y_valid)
encoded_y_test = one_hot_encode(y_test)

In [5]:
# from keras.utils import to_categorical

# encoded_y_train = to_categorical(y_train)
# encoded_y_valid = to_categorical(y_valid)

In [6]:
encoded_y_train[:10]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

# create model
model = Sequential()
# add model layers
model.add(Conv2D(64, kernel_size = 3, activation = 'relu', input_shape =(28, 28, 1)))
model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
model.add(Flatten())
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(X_train, encoded_y_train, validation_data = (X_valid, encoded_y_valid), epochs = 3)

Train on 45000 samples, validate on 15000 samples
Epoch 1/3
45000/45000 [==============================] - 477s 11ms/step - loss: 11.3445 - acc: 0.2950 - val_loss: 11.3009 - val_acc: 0.2988
Epoch 2/3
45000/45000 [==============================] - 486s 11ms/step - loss: 11.2847 - acc: 0.2998 - val_loss: 11.3751 - val_acc: 0.2943
Epoch 3/3
45000/45000 [==============================] - 470s 10ms/step - loss: 11.3818 - acc: 0.2937 - val_loss: 11.2909 - val_acc: 0.2995


In [8]:
from sklearn.metrics import accuracy_score

# y_test = to_categorical(y_test)

y_pred = model.predict(X_test)
# accuracy_score(y_test, y_pred)

In [9]:
y_pred[:10]
op = np.unravel_index(np.argmax(y_pred, axis=1), y_pred.shape)

In [ ]:
accuracy_score(y_test, op[1])